# wikipediaからフル学習したword2vecモデルを使ってIT用語のテスト

In [ ]:
from pathlib import Path

import pandas as pd
from gensim.models import KeyedVectors, Word2Vec
from sudachipy import dictionary, tokenizer

# パス設定
PROJECT_ROOT = Path.cwd().parent  # notebooks/ の1つ上がプロジェクトルート想定
DATA_DIR = PROJECT_ROOT / "data" / "raw"
MODEL_DIR = PROJECT_ROOT / "models"

MODEL_PATH = MODEL_DIR / "word2vec_wiki40b_full.model"
WORDVEC_PATH = MODEL_DIR / "word2vec_wiki40b_full.wordvectors"

print("PROJECT_ROOT:", PROJECT_ROOT)
print("MODEL_PATH:", MODEL_PATH)
print("WORDVEC_PATH:", WORDVEC_PATH)

# モデル読み込み（フル学習済み Wikipedia Word2Vec）
if MODEL_PATH.exists():
    model = Word2Vec.load(str(MODEL_PATH))
    wv = model.wv
    print("Word2Vec model loaded (with training state)")
elif WORDVEC_PATH.exists():
    wv = KeyedVectors.load(str(WORDVEC_PATH), mmap="r")
    model = None
    print("KeyedVectors loaded (word vectors only)")
else:
    raise FileNotFoundError("Word2Vecモデルファイルが見つかりません")

print("vocab size:", len(wv))

PROJECT_ROOT: /Users/aokidai/dev/research
MODEL_PATH: /Users/aokidai/dev/research/models/word2vec_wiki40b_full.model
WORDVEC_PATH: /Users/aokidai/dev/research/models/word2vec_wiki40b_full.wordvectors
Word2Vec model loaded (with training state)
vocab size: 393381


In [8]:
# Sudachi による形態素解析ユーティリティ

# full辞書を使用して辞書オブジェクトを生成
_sudachi_tokenizer = dictionary.Dictionary(dict_type="full").create()

_SPLIT_MODE_MAP = {
    "A": tokenizer.Tokenizer.SplitMode.A,
    "B": tokenizer.Tokenizer.SplitMode.B,
    "C": tokenizer.Tokenizer.SplitMode.C,
}


def tokenize_japanese(text: str, mode: str = "C"):
    """表層形ベースで日本語テキストをトークン化"""
    if not isinstance(text, str) or not text:
        return []
    split_mode = _SPLIT_MODE_MAP.get(mode.upper(), _SPLIT_MODE_MAP["C"])
    return [m.surface() for m in _sudachi_tokenizer.tokenize(text, split_mode)]


def extract_content_words(
    text: str, mode: str = "C", pos_prefix=("名詞", "動詞", "形容詞")
):
    """内容語（名詞・動詞・形容詞など）の基本形を抽出"""
    if not isinstance(text, str) or not text:
        return []
    split_mode = _SPLIT_MODE_MAP.get(mode.upper(), _SPLIT_MODE_MAP["C"])
    tokens = []
    for m in _sudachi_tokenizer.tokenize(text, split_mode):
        pos = m.part_of_speech()
        if not pos or pos[0] not in pos_prefix:
            continue
        base = m.dictionary_form()
        tokens.append(base)
    return tokens

/var/folders/rb/64gq3h_s05lg4ctppr9f3wxh0000gn/T/ipykernel_84194/3515428925.py:4: DeprecationWarning: Parameter dict_type of Dictionary() is deprecated, use dict instead
  _sudachi_tokenizer = dictionary.Dictionary(dict_type="full").create()


In [9]:
# IT用語の類似語テスト


def show_similar_words(term: str, topn: int = 10):
    if term not in wv.key_to_index:
        print(f"'{term}' は語彙に含まれていません")
        # 辞書形・小文字化など軽い正規化も試す
        cand = term.lower()
        if cand != term and cand in wv.key_to_index:
            print(f"→ '{cand}' が見つかったためこちらで検索します")
            term_ = cand
        else:
            return
    else:
        term_ = term

    print(f"[query] {term_}")
    for w, score in wv.most_similar(term_, topn=topn):
        print(f"  {w}\t{score:.4f}")


example_it_terms = [
    "AWS",
    "Docker",
    "Kubernetes",
    "React",
    "Vue",
    "Python",
    "クラウド",
    "インフラ",
    "サーバー",
    "機械学習",
]

for term in example_it_terms:
    print("=" * 40)
    show_similar_words(term)
    print()

[query] AWS
  共同交戦能力	0.6549
  tadixs	0.6532
  ssds	0.6531
  Ballistic Missile Defense Syetem	0.6530
  acds	0.6463
  RWR	0.6430
  wds	0.6381
  stacos	0.6378
  データプロセシング	0.6369
  aswcs	0.6363

[query] Docker
  edirectory	0.8612
  Java Servlet	0.8611
  Apache Tomcat	0.8590
  starteam	0.8571
  Google App Engine	0.8551
  データベースサーバー	0.8519
  idempiere	0.8502
  uniface	0.8502
  zodb	0.8492
  openvz	0.8488

'Kubernetes' は語彙に含まれていません
→ 'kubernetes' が見つかったためこちらで検索します
[query] kubernetes
  edirectory	0.8701
  バックグラウンドプロセス	0.8651
  シングルサインオン	0.8616
  CPAN	0.8581
  netfilter	0.8553
  WMI	0.8553
  LDAP	0.8535
  javabeans	0.8512
  firewalld	0.8510
  myBatis	0.8503

'React' は語彙に含まれていません

[query] Vue
  Nemerle	0.7319
  Google App Engine	0.7283
  scripts	0.7274
  dvia	0.7269
  plugin	0.7235
  workbench	0.7228
  Impress	0.7223
  insights	0.7220
  platforms	0.7214
  ebean	0.7186

[query] Python
  python	0.8260
  Perl	0.8029
  OCaml	0.7714
  NumPy	0.7577
  シェルスクリプト	0.7570
  bcpl	0.7563
  stackless	0.7563
  

In [10]:
# data/raw のCSV読み込み
PROJECT_CSV = DATA_DIR / "project.csv"
USER_WORK_HISTORIES_CSV = DATA_DIR / "user_work_histories.csv"

print("PROJECT_CSV:", PROJECT_CSV)
print("USER_WORK_HISTORIES_CSV:", USER_WORK_HISTORIES_CSV)

project_df = pd.read_csv(PROJECT_CSV)
user_work_histories_df = pd.read_csv(USER_WORK_HISTORIES_CSV)

project_df.head(3)

PROJECT_CSV: /Users/aokidai/dev/research/data/raw/project.csv
USER_WORK_HISTORIES_CSV: /Users/aokidai/dev/research/data/raw/user_work_histories.csv


,id,name,description,desired_role,start_date,end_date,HTML,CSS,JavaScript,TypeScript,...,GCP,Docker,Kubernetes,MySQL,PostgreSQL,MongoDB,Git,Linux,Agile,Scrum
0,1,旅行予約システム設計,当プロジェクトでは運用保守、テスト、デプロイ、ドキュメント作成、自動テスト導入やインフラ構築...,QAエンジニア,2024-04-09,NaN,0,0,0,0,...,0,0,0,0,0,0,1,1,1,0
1,2,予約管理システム設計,現プロジェクトにおいてデータベース設計やデプロイなどに取り組んでいる。\n[実施内容]\nデ...,フルスタックエンジニア,2024-12-22,NaN,1,1,1,1,...,0,1,1,0,0,0,1,1,0,0
2,3,ブロックチェーン開発実装,要件定義、要件分析、パフォーマンス改善、自動テスト導入、クラウド移行やセキュリティ強化などの...,フルスタックエンジニア,2025-05-20,NaN,1,1,1,0,...,0,0,0,1,1,1,1,1,0,0


In [11]:
# user_work_histories の先頭も確認
user_work_histories_df.head(3)

,id,user_id,history_id,project_name,description,role,start_date,end_date,HTML,CSS,...,GCP,Docker,Kubernetes,MySQL,PostgreSQL,MongoDB,Git,Linux,Agile,Scrum
0,1,1,1,スマホアプリ開発,SNSアプリ開発\n[主な担当]\nドキュメント作成\nテスト実施\nインフラ構築\n技術検証,プログラマー,2016-11-28,NaN,1,0,...,1,0,0,0,1,0,0,0,1,0
1,2,1,2,医療情報システム開発,オンプレミス環境からAWSへのクラウド移行を主導しました。 新技術の調査やPoCを行い、アー...,技術リーダー,2016-04-17,2018-01-18,1,0,...,1,0,0,0,1,0,0,0,1,0
2,3,1,3,データ分析基盤の構築,アジャイル手法で開発を進め、定期的なスプリントレビューで改善を行いました。 自動テストの導入...,QAエンジニア,2021-09-27,2022-10-04,1,0,...,1,0,0,0,1,0,0,0,1,0


In [12]:
# 説明文をSudachiでトークン化し、Word2Vecの語彙カバレッジを確認

TEXT_COLS_PROJECT = ["description"]
TEXT_COLS_UWH = ["description"]


def get_tokens_from_df(df: pd.DataFrame, text_cols):
    tokens = []
    for col in text_cols:
        if col not in df.columns:
            continue
        for text in df[col].dropna().astype(str):
            tokens.extend(extract_content_words(text))
    return tokens


project_tokens = get_tokens_from_df(project_df, TEXT_COLS_PROJECT)
uwh_tokens = get_tokens_from_df(user_work_histories_df, TEXT_COLS_UWH)

print("project tokens:", len(project_tokens))
print("user_work_histories tokens:", len(uwh_tokens))

unique_tokens = set(project_tokens) | set(uwh_tokens)

in_vocab = [t for t in unique_tokens if t in wv.key_to_index]
out_vocab = [t for t in unique_tokens if t not in wv.key_to_index]

print("unique tokens:", len(unique_tokens))
print("in_vocab:", len(in_vocab))
print("out_vocab:", len(out_vocab))
print("coverage:", len(in_vocab) / max(len(unique_tokens), 1))

# サンプルとして、語彙に含まれるIT関連っぽい語をいくつか表示
sample_it_like = [
    t
    for t in in_vocab
    if any(
        k in t
        for k in [
            "クラウド",
            "AWS",
            "Docker",
            "Kubernetes",
            "React",
            "Vue",
            "サーバ",
            "インフラ",
        ]
    )
][:50]
print("sample IT-like tokens in vocab:")
print(sample_it_like)

project tokens: 14621
user_work_histories tokens: 16748
unique tokens: 163
in_vocab: 150
out_vocab: 13
coverage: 0.9202453987730062
sample IT-like tokens in vocab:
['インフラ', 'クラウド', 'AWS', 'Vue']
